# ABD2

Wij gaan tijdens deze opdracht een food-collector simulatie bouwen.
In deze simulatie is het doel van de agents om zo veel mogelijk groen eten op te eten, terwijl ze rood eten vermijden.
We gaan deze simulatie grid-based maken. 

We maken gebruik van de gymnasium API: https://gymnasium.farama.org/

Ter visualisatie gebruiken we pygame.


### Simulation properties:
We beginnen in principe met een grid van 128x128, maar dit kan uitgebreid worden. 
We zullen de implementatie zoveel mogelijk scalable en aanpasbaar implementeren, door hyperparameters aan te maken voor alle belangrijke properties.

- Number of agents
- Grid size (default 128x128)
- Good food to total square ratio
- Good food spawning pattern (maybe)
- Bad food to total square ratio
- Bad food spawning pattern (maybe)
- Episode duration

### Agent properties:
Actions: up, down, left, right, wait
Perception: full information (knows coordinates of other agents, good food, and bad food)
Agent rules: not yet specified





In [1]:
import numpy as np
import pygame
import gymnasium as gym
from gymnasium import spaces
import random
import time
import tcod

pygame 2.1.3.dev8 (SDL 2.0.22, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class agent():
    def __init__(self):
        self.location = [None, None]
        self.points = 0
        self.vision_range = 3
        self.memory = []
        self._action_to_direction = {
            0: np.array([0, 0]),
            1: np.array([1, 0]),
            2: np.array([0, 1]),
            3: np.array([-1, 0]),
            4: np.array([0, -1])
        }
        self.path = []
        self.q_dict = {}

    def move(self, direction):
        """move an agent into a specific direction"""
        
        new_location = self.location + direction
        self.location = list(new_location)
        

    def clip_vision(self, obs):
        """clip the observation of the agent to the vision_range"""

        clipped_obs = {}

        for k in obs.keys():

            clipped_obs[k] = [x for x in obs[k]
                      if abs(self.location[0] - x[0]) <= self.vision_range
                      and abs(self.location[1] - x[1]) <= self.vision_range
                     ]

        return clipped_obs
    
    def get_best_move(self, obs, grid_size):
        """return the best action an agent can take, calculated with a*; returns None if no good food in vision range"""
        
        cost = np.full((self.vision_range*2 + 1, self.vision_range*2 + 1),2)
        
        for k in obs.keys():
            
            new_coords = [[x-self.location[0] + self.vision_range,y-self.location[1]+ self.vision_range]  for x,y in obs[k]]
            
            if k == 'bad_food_locs':
                for x in new_coords:
                    cost[x[0]][x[1]] = 10
                    # it can be worth it for the agent to cross bad food to get to good food with the current point assignment (-1,5) if it's the only path available
            if k == 'good_food_locs':
                for x in new_coords:
                    cost[x[0]][x[1]] = 1
            if k == 'agent_locs':
                for x in new_coords:
                    cost[x[0]][x[1]] = 0
            
            # out of bounds squares get value 0
            for x in np.argwhere(cost != None):
                converted_coord = (np.array(x) - np.array([self.vision_range,self.vision_range]) + self.location)
                if converted_coord[0] > grid_size -1 or converted_coord[0] < 0 or converted_coord[1] > grid_size -1 or converted_coord[1] < 0:
                    cost[x[0]][x[1]] = 0
                    
                
        good_food_converted_locs = list(zip(*np.where(cost == 1)))
        #print(np.rot90(cost, k=1, axes=(0, 1)))
    
        cost = cost.astype(np.int8) #changed type, now it works
        graph = tcod.path.SimpleGraph(cost = cost, cardinal =1, diagonal = 0)
        pf = tcod.path.Pathfinder(graph)
        pf.add_root((self.vision_range, self.vision_range))
        pf.resolve()
        
           
        best_move = None
        
        # calculate the best paths to all available good_food in vision_range
        paths = [pf.path_to(cl)[1:].tolist() for cl in good_food_converted_locs if len(pf.path_to(cl)[1:].tolist()) != 0]

        self.path = []
        if len(paths) > 0:
            shortest_path_index = np.argmin([len(x) for x in paths])
            for x in paths[shortest_path_index]:
                self.path.append(list(np.array(x) - np.array([-self.location[0] + self.vision_range,-self.location[1]+ self.vision_range])))
            best_move = list(np.array(paths[shortest_path_index][0]) - np.array([self.vision_range, self.vision_range]))
            best_move = [x[0] for x in list(self._action_to_direction.items()) if x[1][0] == best_move[0] and x[1][1] == best_move[1]][0]
            
            return best_move
        
        return None
            

        
        
    def pick_action(self,obs, grid_size):
        
                
        best_move = self.get_best_move(obs, grid_size)

        viable_actions = list(self._action_to_direction.keys())

        other_agent_locations = [x for x in obs['agent_locs'] if x != self.location]

        # Rule 1: agents can't move to a space occupied by another agent
        viable_actions = [v for v in viable_actions if list(self.location +  self._action_to_direction[v]) not in other_agent_locations]
        # Rule 2: agents can't move out of bounds
        viable_actions = [v for v in viable_actions
                          if list(self.location +  self._action_to_direction[v])[0] >= 0
                          and list(self.location +  self._action_to_direction[v])[1] >= 0
                          and list(self.location +  self._action_to_direction[v])[0] < grid_size
                          and list(self.location +  self._action_to_direction[v])[1] < grid_size]
        

        #Rule 3: never step on bad food (while exploring)
        viable_actions = [k for k,v in {v:list(self.location + self._action_to_direction[v]) for v in viable_actions}.items()
                          if v not in obs['bad_food_locs']]
    
     
        
        #Rule 4: prioritize undiscovered squares over discovered squares without good food in vision range
        # checks the last 15 coordinates (15 has the lowest average steps for sim completion)
        viable_actions_higher_prio = [k for k,v in {v:list(self.location + self._action_to_direction[v]) for v in viable_actions}.items()
                          if v not in [x['agent_location'] for x in self.memory[-15:] if len(x['agent_obs']['good_food_locs'])==0]]
         
        # unstuck the agent if it gets caught in a loop
        action_history = [x['agent_action'] for x in self.memory][-10:]
        if len(action_history) >= 10:
            if len(list(set(action_history))) == 2:
                if len(list(set([action_history[l] for l in range(len(action_history)) if l % 2 == 0]))) == 1 and len(list(set([action_history[l] for l in range(len(action_history)) if l % 2 == 1]))) == 1:
                   
                    return random.choice(viable_actions)
    
        
        if best_move == None:
            if len(viable_actions_higher_prio) > 0:
                action = random.choice(viable_actions_higher_prio)
            else:
                action = random.choice(viable_actions)

        else:

            action = best_move
        
        return action
    
        
        
    def run(self, obs, grid_size):
        
        # clip the observation to the agents vision
        clipped_obs = self.clip_vision(obs)
        
        # decide which action to take based on the previous observation
        action = self.pick_action(clipped_obs, grid_size)
        
        # append the new information to the agent's memory
        self.memory.append({'agent_action':action, "agent_obs":clipped_obs, "agent_location":self.location})
        
        
        return action
        



In [3]:
class foodCollectorEnv(gym.Env):
    metadata = {"render_modes": ["human", None], "render_fps": 4}

    def __init__(self, render_mode=None, grid_size=128, game_duration=200, agents=1, good_food_ratio=0.1,
                 bad_food_ratio=0.1):
        self.grid_size = grid_size
        self.window_size = 1028
        self.game_duration = game_duration * agents
        self.current_step = 0
        self.agents = [agent() for _ in range(agents)]
        self.good_food_ratio = good_food_ratio
        self.bad_food_ratio = bad_food_ratio
        self.good_food_points = 5
        self.bad_food_points = -1
        self.render_mode = render_mode
        self.window = None
        self.grid = np.ones((self.grid_size, self.grid_size))


        self.observation_space = spaces.Dict({
            'agent_locs': spaces.Sequence(spaces.MultiDiscrete([self.grid_size - 1, self.grid_size - 1])),
            'good_food_locs': spaces.Sequence(spaces.MultiDiscrete([self.grid_size - 1, self.grid_size - 1])),
            'bad_food_locs': spaces.Sequence(spaces.MultiDiscrete([self.grid_size - 1, self.grid_size - 1])),
        })

        self.action_space = spaces.MultiDiscrete(5)

        self._action_to_direction = {
            0: np.array([0, 0]),
            1: np.array([1, 0]),
            2: np.array([0, 1]),
            3: np.array([-1, 0]),
            4: np.array([0, -1])
        }

        self.good_food_locations = []
        self.bad_food_locations = []

    def get_obs(self):
        return {
            "agent_locs": [agent.location for agent in self.agents],
            "good_food_locs": self.good_food_locs,
            "bad_food_locs": self.bad_food_locs
        }

    def reset(self, seed=None, options=None):

        super().reset(seed=seed)

        self.agent_locs = []
        self.good_food_locs = []
        self.bad_food_locs = []


        # place agents
        while True:
            if len(self.agent_locs) >= len(self.agents):
                break
            loc = [random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1)]
            if loc not in self.agent_locs:
                self.agent_locs.append(loc)


        # place good food
        while True:
            if len(self.good_food_locs) >= round(self.good_food_ratio * self.grid_size * self.grid_size):
                break
            loc = [random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1)]
            if loc not in self.agent_locs and loc not in self.good_food_locs and loc not in self.bad_food_locs:
                self.good_food_locs.append(loc)

        #place bad food
        while True:
            if len(self.bad_food_locs) >= round(self.bad_food_ratio * self.grid_size * self.grid_size):
                break
            loc = [random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1)]
            if loc not in self.agent_locs and loc not in self.good_food_locs and loc not in self.bad_food_locs:
                self.bad_food_locs.append(loc)

        for i, agent in enumerate(self.agents):
            agent.location = self.agent_locs[i]
        
        observation = self.get_obs()

        if self.render_mode == "human":
            self.render_frame()

        info = {"agent_points": [agent.points for agent in self.agents]}



        return observation, info

    def step(self, action, agent_index):
        
        direction = self._action_to_direction[action[0]]
        
        self.agents[agent_index].move(direction)

        if self.agents[agent_index].location in self.good_food_locs:
            self.good_food_locs.remove(self.agents[agent_index].location)
            self.agents[agent_index].points += self.good_food_points

        if self.agents[agent_index].location in self.bad_food_locs:
            self.bad_food_locs.remove(self.agents[agent_index].location)
            self.agents[agent_index].points += self.bad_food_points

        terminated = self.current_step >= self.game_duration or (len(self.get_obs()['good_food_locs']) == 0 and self.current_step > 0)
        reward = 0 # niet relevant voor nu
        observation = self.get_obs()

        if self.render_mode == "human":
            self.render_frame()
        info = {"agent_points": [agent.points for agent in self.agents]}

        self.current_step += 1

        return observation, reward, terminated, info

    def _is_empty(self, loc):

        for agent in self.agents:
            if sorted(agent.location) == sorted(loc):
                return False

        return True

    def render(self):
        if self.render_mode == "rgb_array":
            return self.render_frame()

    def render_frame(self):
        pygame.font.init()
        font = pygame.font.Font(None, 24)
        scoreboard_width = 0.2

        if self.window is None and self.render_mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode(
                (self.window_size + self.window_size * scoreboard_width, self.window_size)
            )

        canvas = pygame.Surface((self.window_size + self.window_size * scoreboard_width, self.window_size))
        canvas.fill((204, 255, 229))
        pix_square_size = (
                self.window_size / self.grid_size
        )

        #draw the background for the scoreboard
        scoreboard_bg = pygame.Surface((self.window_size*scoreboard_width,self.window_size+self.window_size*scoreboard_width))
        scoreboard_bg.fill((255, 255, 255))
        scoreboard_bg.set_alpha(255)
        canvas.blit(scoreboard_bg, (self.window_size, 0))
            
        for i, agent in enumerate(self.agents):
            #draw agent picture
            deer_image = pygame.image.load("deer.png").convert_alpha()
            deer_image = pygame.transform.scale(deer_image, (pix_square_size, pix_square_size))
            canvas.blit(deer_image, (agent.location[0] * pix_square_size, (self.grid_size - 1 - agent.location[1]) * pix_square_size))

            #try to draw vision_range, a bit awkward imo
            fov = pygame.Surface((pix_square_size * (agent.vision_range*2+1), pix_square_size * (agent.vision_range*2+1)))
            fov.set_alpha(128)
            fov.fill((137,137,137))
            canvas.blit(fov, ((agent.location[0] * pix_square_size)-pix_square_size*agent.vision_range, ((self.grid_size - 1 - agent.location[1]) * pix_square_size) - pix_square_size * agent.vision_range))

            
            score_text = font.render(f'Agent {i} | {agent.points}', True, "black")
            canvas.blit(score_text, (self.window_size + 10 , 10 + (i*50)))
            
            score_text = font.render(f"{i}", True, "white")
            canvas.blit(score_text, (agent.location[0] * pix_square_size, (self.grid_size - 1 - agent.location[1]) * pix_square_size))
            

            #path to good food
            if len(agent.path) > 1:
                for coord in agent.path[1:]:
                    #draw the path
                    path = pygame.Surface((pix_square_size, pix_square_size))
                    path.set_alpha(128)
                    path.fill((255, 0, 0))
                    canvas.blit(path, (coord[0] * pix_square_size, (self.grid_size - 1 - coord[1]) * pix_square_size))
        
        #for i, agent in enumerate(self.agents):
            #scoreboard text
            #score_text = font.render(f'Agent {i} | {agent.points}', True, "black")
            #canvas.blit(score_text, (self.window_size + 10 , 10 + (i*50)))
            
  
        for gfl in self.good_food_locs:
            good_food_image = pygame.image.load("plant.png").convert_alpha()
            good_food_image = pygame.transform.scale(good_food_image, (pix_square_size, pix_square_size))
            canvas.blit(good_food_image, (gfl[0] * pix_square_size, (self.grid_size - 1 - gfl[1]) * pix_square_size))

        for bfl in self.bad_food_locs:
            bad_food_image = pygame.image.load("evil_plant.png").convert_alpha()
            bad_food_image = pygame.transform.scale(bad_food_image, (pix_square_size, pix_square_size))
            canvas.blit(bad_food_image, (bfl[0] * pix_square_size, (self.grid_size - 1 - bfl[1]) * pix_square_size))

        for x in range(self.grid_size + 1):
            pygame.draw.line(
                canvas,
                color=0,
                start_pos=(0, pix_square_size * x),
                end_pos=(self.window_size, pix_square_size * x),
                width=1,
            )
            pygame.draw.line(
                canvas,
                color=0,
                start_pos=(pix_square_size * x, 0),
                end_pos=(pix_square_size * x, self.window_size),
                width=1,
            )

        if self.render_mode == "human":

            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()

            #self.clock.tick(self.metadata["render_fps"])
        else:
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )


In [4]:
env = foodCollectorEnv(render_mode="human", grid_size=15, game_duration=1000, agents=1, good_food_ratio=0.15,
                       bad_food_ratio=0.4)
env.reset()

observation = env.get_obs()
while True:

    for i in range(len(env.agents)):

        action = [env.agents[i].run(observation, env.grid_size)]
        observation, reward, terminated, info = env.step(action,i)

    if terminated:
        break
pygame.quit()


In [5]:
print(info)
np.argmax(info['agent_points'])
print(f"Agent {np.argmax(info['agent_points'])} wins the game with {max(info['agent_points'])} points")

{'agent_points': [162]}
Agent 0 wins the game with 162 points


In [6]:
lst = [1,10,2,2]

In [7]:
lst[np.argmin(lst)] / sum(lst)

0.06666666666666667

In [8]:
[1- lst[i] / sum(lst) for i in range(len(lst))]

[0.9333333333333333,
 0.33333333333333337,
 0.8666666666666667,
 0.8666666666666667]

In [9]:
[[random.choices(lst, weights = [1- lst[i] / sum(lst) for i in range(len(lst))])[0] for _ in range(1000)].count(lst[x]) for x in range(len(lst))]

[328, 121, 547, 565]

In [10]:
env.action_space.sample()

3

In [11]:
env.current_step

310